In [6]:
import tensorflow as tf
import numpy as np
import cv2
import os, glob
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [5]:
# --- Helper function for plotting ---
def plot_history(history, title, save_path):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title(f'{title} - Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title(f'{title} - Loss')

    plt.savefig(save_path)
    plt.show()

**This notebook was created on kaggle. Make sure the dataset UTKFace from Sanjaya Subedi is added as input already in kaggle.**

In [7]:
# CONFIG

DATASET_PATH = "/kaggle/input/utkface-new"   # Kaggle dataset mount
MODEL_PATH = "/kaggle/working/gender_model.h5"
IMG_WIDTH_AGE_GENDER = 128
IMG_HEIGHT_AGE_GENDER = 128
BATCH_SIZE = 32

In [8]:
# Get file paths and labels - I am doing it seperately here since I want to make sure all the files are properly detected
def get_gender_paths_and_labels():
    print("Scanning UTKFace for gender data...")
    paths, genders = [], []
    all_files = glob.glob(os.path.join(DATASET_PATH, "**/*.jpg"), recursive=True)
    print(f"Found {len(all_files)} images")

    for filepath in all_files:
        try:
            parts = os.path.basename(filepath).split('_')
            gender = int(parts[1])  # 0 = male, 1 = female
            paths.append(filepath)
            genders.append(gender)
        except:
            continue
    return paths, genders

In [9]:
# Creating the tf pipeling, well sort of.

def parse_image_gender(filename, gender):
    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_WIDTH_AGE_GENDER, IMG_HEIGHT_AGE_GENDER])
    image = image / 255.0
    return image, gender

def create_dataset(paths, labels, training=False):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(parse_image_gender, num_parallel_calls=tf.data.AUTOTUNE)
    if training:
        ds = ds.shuffle(1000)
    return ds.batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)


In [10]:
# for building gender model

def build_gender_model():
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(IMG_WIDTH_AGE_GENDER, IMG_HEIGHT_AGE_GENDER, 3),
        include_top=False, weights='imagenet'
    ) # building on top of MobileNetV2
    base_model.trainable = False  # freeze feature extractor
    
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # basics - b_crossentropy since we have 0 and 1 only
    return model


In [11]:
# fun part - training and evaluation

print("--- Training and Evaluating Gender Model ---")
paths, genders = get_gender_paths_and_labels()

# Stratified split
X_train_val, X_test, y_train_val, y_test = train_test_split(
    paths, genders, test_size=0.2, random_state=42, stratify=genders
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val
)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

# Create datasets
train_ds = create_dataset(X_train, y_train, training=True)
val_ds = create_dataset(X_val, y_val)
test_ds = create_dataset(X_test, y_test)

# Build + Train
gender_model = build_gender_model()
history = gender_model.fit(
    train_ds,
    epochs=15,
    validation_data=val_ds,
    verbose=1
)


# Save Model
gender_model.save(MODEL_PATH)
print(f"✅ Gender model saved at {MODEL_PATH}")

# Saving a .tflite version as well

loaded = tf.keras.models.load_model(MODEL_PATH)  
converter = tf.lite.TFLiteConverter.from_keras_model(loaded)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_from_h5 = converter.convert()
open("gender_model_from_saved.tflite", "wb").write(tflite_from_h5)
print("Saved gender_model_from_saved.tflite")

--- Training and Evaluating Gender Model ---
Scanning UTKFace for gender data...
Found 66976 images
Train: 40185, Val: 13395, Test: 13396


I0000 00:00:1756594527.442555      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1756594527.443276      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15


I0000 00:00:1756594539.090525     108 service.cc:148] XLA service 0x7c88bc0527d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756594539.091858     108 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1756594539.091877     108 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1756594540.103558     108 cuda_dnn.cc:529] Loaded cuDNN version 90300


   4/1256 ━━━━━━━━━━━━━━━━━━━━ 55s 44ms/step - accuracy: 0.5384 - loss: 1.0740 

I0000 00:00:1756594544.577911     108 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1256/1256 ━━━━━━━━━━━━━━━━━━━━ 106s 73ms/step - accuracy: 0.7627 - loss: 0.4934 - val_accuracy: 0.8245 - val_loss: 0.3695
Epoch 2/15
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - accuracy: 0.8186 - loss: 0.3912 - val_accuracy: 0.8405 - val_loss: 0.3424
Epoch 3/15
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.8331 - loss: 0.3635 - val_accuracy: 0.8458 - val_loss: 0.3316
Epoch 4/15
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.8404 - loss: 0.3482 - val_accuracy: 0.8514 - val_loss: 0.3245
Epoch 5/15
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.8510 - loss: 0.3302 - val_accuracy: 0.8596 - val_loss: 0.3065
Epoch 6/15
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.8587 - loss: 0.3192 - val_accuracy: 0.8625 - val_loss: 0.3067
Epoch 7/15
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.8679 - loss: 0.3003 - val_accuracy: 0.8785 - val_loss: 0.2839
Epoch 8/15
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.8722 - loss: 0.2

W0000 00:00:1756594922.455863      36 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1756594922.455901      36 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1756594922.602236      36 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled


In [12]:
# evaluation

print("\n--- Evaluating on Test Set ---")
y_pred_probs = gender_model.predict(test_ds)
y_pred = (y_pred_probs > 0.5).astype("int32")
y_true = np.concatenate([y for _, y in test_ds], axis=0)

print(classification_report(y_true, y_pred, target_names=['Male', 'Female']))



--- Evaluating on Test Set ---
419/419 ━━━━━━━━━━━━━━━━━━━━ 28s 58ms/step
              precision    recall  f1-score   support

        Male       0.91      0.91      0.91      6706
      Female       0.91      0.91      0.91      6690

    accuracy                           0.91     13396
   macro avg       0.91      0.91      0.91     13396
weighted avg       0.91      0.91      0.91     13396

